In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import csv
import os

# 指定文件夾路徑和CSV文件名稱
folder_path = "C:\\Users\\User\\Desktop\\賽馬"
csv_file_name = "2020-2023賽馬賽事.csv"

# 確保文件夾存在，如果不存在則創建它
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# 組合完整的CSV文件路徑
csv_file_path = os.path.join(folder_path, csv_file_name)

# 使用請求headers
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.3"}

def fetch_race_data(race_url, race_date):
    response_race = requests.get(race_url, headers=headers)


    if response_race.status_code != 200:
        print(f"無法獲取比賽信息，比賽URL: {race_url}")
        return []

    response_race.encoding = 'EUC-JP'
    soup_race = BeautifulSoup(response_race.text, 'html.parser')
    race_data_list = []

    race_info = soup_race.find("div", {"class": "mainrace_data fc"})
    if not race_info:
        print(f"無法獲取比賽信息，比賽URL: {race_url}")
        return []

    race_name = race_info.find("h1").text.strip()
    span_text = race_info.find("span").text.strip().replace("\xa0", " ").replace("/", "、")
    split_text = span_text.split("、")
    distance = split_text[0] if len(split_text) > 0 else ""
    weather = split_text[1] if len(split_text) > 1 else ""
    race_number = soup_race.find("dt").text.strip()



    race_place_links = soup_race.select("ul.race_place.fc a.active")
    race_place = race_place_links[0].text if race_place_links else ""

    race_table = soup_race.find("table", {"class": "race_table_01 nk_tb_common"})
    if race_table:
        rows = race_table.find_all("tr")
        for row in rows[1:]:
            cols = row.find_all("td")
            if len(cols) >= 21:
                race_data_list.append({
                    '賽事時間': race_date,
                    '地點': race_place,
                    '場名': race_name,
                    '場次': race_number,
                    '距離': distance,
                    '天氣': weather,
                    '名次': cols[0].text.strip(),
                    '馬名': cols[3].text.strip(),
                    '性別年紀': cols[4].text.strip(),
                    '馬重量': cols[14].text.strip(),
                    '騎手': cols[6].text.strip(),
                    '重量': cols[5].text.strip(),
                    '抵達時間': cols[7].text.strip(),
                    '單柱賠率': cols[12].text.strip(),
                    '獎金': cols[20].text.strip()
                })
    return race_data_list

with open(csv_file_path, mode='w', newline='', encoding='utf-8-sig') as csv_file:
    fieldnames = ['賽事時間', '地點', '場名', '場次', '距離', '天氣', '名次', '馬名', '性別年紀', '馬重量', '騎手', '重量', '抵達時間', '單柱賠率', '獎金']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    start_date = datetime(2020, 1, 1)
    end_date = datetime(2023, 10, 1)
    
    current_date = start_date
    while current_date <= end_date:
        formatted_date = current_date.strftime("%Y%m%d")
        date_url = f"https://db.netkeiba.com/race/list/{formatted_date}/"
        
        response_date = requests.get(date_url, headers=headers)
        if response_date.status_code != 200:
            print(f"無法獲取日期信息，日期URL: {date_url}")
            current_date += timedelta(days=1)
            continue

        soup_date = BeautifulSoup(response_date.text, 'html.parser')
        race_links = [a["href"] for a in soup_date.select(".race_top_data_info a") if "/movie/" not in a["href"]]


        if not race_links:
            print(f"{formatted_date} 沒有賽事。")
            current_date += timedelta(days=1)
            continue

        for race_link in race_links:
            race_data = fetch_race_data(f"https://db.netkeiba.com{race_link}", formatted_date)
            for item in race_data:
                writer.writerow(item)
        print(f"{formatted_date} 完成。")
        current_date += timedelta(days=1)


20200101 沒有賽事。
20200102 沒有賽事。
20200103 沒有賽事。
20200104 沒有賽事。
20200105 完成。
20200106 完成。
20200107 沒有賽事。
20200108 沒有賽事。
20200109 沒有賽事。
20200110 沒有賽事。
20200111 完成。
20200112 完成。
20200113 完成。
20200114 沒有賽事。
20200115 沒有賽事。
20200116 沒有賽事。
20200117 沒有賽事。
20200118 完成。
20200119 完成。
20200120 沒有賽事。
20200121 沒有賽事。
20200122 沒有賽事。
20200123 沒有賽事。
20200124 沒有賽事。
20200125 完成。
20200126 完成。
20200127 沒有賽事。
20200128 沒有賽事。
20200129 沒有賽事。
20200130 沒有賽事。
20200131 沒有賽事。
20200201 完成。
20200202 完成。
20200203 沒有賽事。
20200204 沒有賽事。
20200205 沒有賽事。
20200206 沒有賽事。
20200207 沒有賽事。
20200208 完成。
20200209 完成。
20200210 沒有賽事。
20200211 沒有賽事。
20200212 沒有賽事。
20200213 沒有賽事。
20200214 沒有賽事。
20200215 完成。
20200216 完成。
20200217 沒有賽事。
20200218 沒有賽事。
20200219 沒有賽事。
20200220 沒有賽事。
20200221 沒有賽事。
20200222 完成。
20200223 完成。
20200224 沒有賽事。
20200225 沒有賽事。
20200226 沒有賽事。
20200227 沒有賽事。
20200228 沒有賽事。
20200229 完成。
20200301 完成。
20200302 沒有賽事。
20200303 沒有賽事。
20200304 沒有賽事。
20200305 沒有賽事。
20200306 沒有賽事。
20200307 完成。
20200308 完成。
20200309 沒有賽事。
2020031

In [9]:

import pandas as pd
df = pd.read_csv('2020-2023賽馬賽事_1.csv')
print(df.isna().sum())

# missing_race_place_dates = df[df['地點'].isna()]['賽事時間']
# print(missing_race_place_dates)

missing_race_place_dates = df[df['地點'].isna()]['賽事時間'].unique()
print("缺失地點的日期：")
if len(missing_race_place_dates) > 0:
    for date in missing_race_place_dates:
        print(date)
else:
    print("nothing")

賽事時間       0
地點         0
場名         0
場次         0
距離         0
天氣         0
名次         0
馬名         0
性別年紀       0
馬重量        0
騎手         0
重量         0
抵達時間    1400
單柱賠率       0
獎金         0
dtype: int64
缺失地點的日期：
nothing


In [39]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('2020-2023賽馬賽事.csv')

# 定义特定的赛事时间和填充值
fill_value = "中山"

# 使用 loc 选择特定赛事时间的行，并填充地点列的缺失值为"中山"
df.loc[(df['賽事時間'] == 20200329) & (df['地點'].isna()), '地點'] ="中山"
df.loc[(df['賽事時間'] == 20200331) & (df['地點'].isna()), '地點'] ="中山"
df['獎金'].fillna(0, inplace=True)
# 选择指定日期的前16筆資料
filtered_data1 = df[df['賽事時間'] == 20200329].head(10)
filtered_data2 = df[df['賽事時間'] == 20200331].head(10)
# 打印出指定日期的前16筆資料
print(filtered_data1)
print(filtered_data2)

# 保存修改後的DataFrame到新的CSV文件
df.to_csv('2020-2023賽馬賽事_1.csv', index=False, encoding='utf-8-sig')


           賽事時間  地點     場名   場次        距離        天氣  名次         馬名 性別年紀  \
12114  20200329  中山  3歳未勝利  1 R  ダ右1800m    天候 : 雨    1    ピュアブレンド   牝3   
12115  20200329  中山  3歳未勝利  1 R  ダ右1800m    天候 : 雨    2   ジュールサイクル   牝3   
12116  20200329  中山  3歳未勝利  1 R  ダ右1800m    天候 : 雨    3   ストリートピアノ   牝3   
12117  20200329  中山  3歳未勝利  1 R  ダ右1800m    天候 : 雨    4  プレシャススパート   牝3   
12118  20200329  中山  3歳未勝利  1 R  ダ右1800m    天候 : 雨    5    ルージュキッス   牝3   
12119  20200329  中山  3歳未勝利  1 R  ダ右1800m    天候 : 雨    6     ラブキーセキ   牝3   
12120  20200329  中山  3歳未勝利  1 R  ダ右1800m    天候 : 雨    7    キアナポラリス   牝3   
12121  20200329  中山  3歳未勝利  1 R  ダ右1800m    天候 : 雨    8    カイエトゥール   牝3   
12122  20200329  中山  3歳未勝利  1 R  ダ右1800m    天候 : 雨    9     カゼニノッテ   牝3   
12123  20200329  中山  3歳未勝利  1 R  ダ右1800m    天候 : 雨   10    フクノフェザン   牝3   

            馬重量    騎手    重量    抵達時間   單柱賠率     獎金  
12114    486(0)  木幡育也  52.0  1:55.5    4.5  510.0  
12115    448(0)  内田博幸  54.0  1:56.0    5.1  200.0  
12116   458(-4)   

In [36]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('2020-2023賽馬賽事.csv')

# 定义特定的赛事时间
specific_date = 20200329  # 修改为您所需的赛事时间

# 选择指定日期的前16筆資料
filtered_data = df[df['賽事時間'] == specific_date].head(16)

# 打印出指定日期的前16筆資料
print(filtered_data)


           賽事時間   地點     場名   場次        距離        天氣  名次         馬名 性別年紀  \
12114  20200329  NaN  3歳未勝利  1 R  ダ右1800m    天候 : 雨    1    ピュアブレンド   牝3   
12115  20200329  NaN  3歳未勝利  1 R  ダ右1800m    天候 : 雨    2   ジュールサイクル   牝3   
12116  20200329  NaN  3歳未勝利  1 R  ダ右1800m    天候 : 雨    3   ストリートピアノ   牝3   
12117  20200329  NaN  3歳未勝利  1 R  ダ右1800m    天候 : 雨    4  プレシャススパート   牝3   
12118  20200329  NaN  3歳未勝利  1 R  ダ右1800m    天候 : 雨    5    ルージュキッス   牝3   
12119  20200329  NaN  3歳未勝利  1 R  ダ右1800m    天候 : 雨    6     ラブキーセキ   牝3   
12120  20200329  NaN  3歳未勝利  1 R  ダ右1800m    天候 : 雨    7    キアナポラリス   牝3   
12121  20200329  NaN  3歳未勝利  1 R  ダ右1800m    天候 : 雨    8    カイエトゥール   牝3   
12122  20200329  NaN  3歳未勝利  1 R  ダ右1800m    天候 : 雨    9     カゼニノッテ   牝3   
12123  20200329  NaN  3歳未勝利  1 R  ダ右1800m    天候 : 雨   10    フクノフェザン   牝3   
12124  20200329  NaN  3歳未勝利  1 R  ダ右1800m    天候 : 雨   11  アオイサンゴショウ   牝3   
12125  20200329  NaN  3歳未勝利  1 R  ダ右1800m    天候 : 雨   12  サフランハーモニー   牝3   
12126  20200

     ---------------------------------------- 96.1/96.1 kB 1.4 MB/s eta 0:00:00
     -------------------------------------- 242.2/242.2 kB 3.7 MB/s eta 0:00:00
     -------------------------------------- 133.1/133.1 kB 7.7 MB/s eta 0:00:00
     ---------------------------------------- 97.9/97.9 kB 5.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
